### Первичное создание и наполнение таблиц базы (с помощью psycopg)
#### Несколько ниже тоже сделано с помощью sqlalchemy

In [8]:
%%capture
!pip install psycopg2 openai ipython-sql python-dotenv

In [28]:
from dotenv import load_dotenv
import os

# Загрузка переменных окружения
load_dotenv()

# Получение значений из .env файла
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Формирование строки подключения
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

print("Строка подключения успешно создана.")


Строка подключения успешно создана.


In [30]:
import openai
import psycopg2

# Подключение к PostgreSQL
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()

# Тестовый запрос к базе данных
cursor.execute("SELECT * FROM students;")
users = cursor.fetchall()
for id in users:
    print(id)

# Закрытие соединения
cursor.close()
conn.close()


(1, {'city': 'Москва', 'group': '101', 'hobbies': ['программирование', 'шахматы'], 'last_name': 'Иванов', 'birth_year': 1998, 'first_name': 'Иван', 'patronymic': 'Иванович'})
(2, {'city': 'Санкт-Петербург', 'group': '102', 'hobbies': ['танцы', 'музыка'], 'last_name': 'Петров', 'birth_year': 1999, 'first_name': 'Петр', 'patronymic': 'Алексеевич'})
(3, {'city': 'Архангельск', 'group': '103', 'hobbies': ['искусство', 'туризм'], 'last_name': 'Смирнова', 'birth_year': 2001, 'first_name': 'Ольга', 'patronymic': 'Алексеевна'})


In [ ]:
# Создание таблицы users
import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    patronymic VARCHAR(50),
    birth_year INT CHECK (birth_year BETWEEN 1900 AND 2100),
    city VARCHAR(100),
    role VARCHAR(10) CHECK (role IN ('student', 'teacher')) NOT NULL
);
"""

# Выполнение SQL-запроса
try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Таблица 'users' успешно создана.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")
    
cursor.close()
conn.close()

Ошибка выполнения запроса: no schema has been selected to create in
LINE 2: CREATE TABLE IF NOT EXISTS users (
                                   ^



Таблица 'users' успешно создана.
Структура таблицы 'users':
('id', 'integer')
('birth_year', 'integer')
('last_name', 'character varying')
('city', 'character varying')
('role', 'character varying')
('patronymic', 'character varying')
('first_name', 'character varying')

In [ ]:
import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
create_tables_query = """
CREATE TABLE disciplines (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    description TEXT
);

-- Таблица расписания занятий
CREATE TABLE sessions (
    id SERIAL PRIMARY KEY,
    discipline_id INT REFERENCES disciplines(id),
    type VARCHAR(20) CHECK (type IN ('lecture', 'seminar', 'exam')) NOT NULL,
    speaker_id INT REFERENCES users(id),
    start_time TIMESTAMP NOT NULL,
    end_time TIMESTAMP NOT NULL,
    system VARCHAR(50),
    room_id BIGINT
);

-- Таблица связей студентов с занятиями
CREATE TABLE students_sessions (
    id SERIAL PRIMARY KEY,
    student_id INT REFERENCES users(id),
    session_id INT REFERENCES sessions(id)
);

-- Таблица видеозаписей
CREATE TABLE videos (
    id SERIAL PRIMARY KEY,
    session_id INT REFERENCES sessions(id),
    file_path VARCHAR(255) NOT NULL,
    upload_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Таблица аннотаций видеозаписей
CREATE TABLE video_annotations (
    id SERIAL PRIMARY KEY,
    video_id INT REFERENCES videos(id),
    timestamp INTERVAL NOT NULL,
    content TEXT
);

-- Таблица раздаточных материалов
CREATE TABLE materials (
    id SERIAL PRIMARY KEY,
    session_id INT REFERENCES sessions(id),
    file_path VARCHAR(255) NOT NULL,
    upload_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Таблица аннотаций материалов
CREATE TABLE material_annotations (
    id SERIAL PRIMARY KEY,
    material_id INT REFERENCES materials(id),
    page_number INT NOT NULL,
    content TEXT
);
"""
# Выполнение SQL-запроса
try:
    cursor.execute(create_tables_query)
    conn.commit()
    print("Таблицы успешно созданы.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")

cursor.close()
conn.close()

In [8]:

import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
# Добавление данных в таблицы users, disciplines, sessions
insert_query = """
-- Наполнение таблицы users
INSERT INTO users (first_name, last_name, patronymic, birth_year, city, role) VALUES
('Иван', 'Иванов', 'Иванович', 1998, 'Москва', 'student'),
('Петр', 'Петров', 'Петрович', 1985, 'Саранск', 'student'),
('Игорь', 'Павлов', 'Алексеевич', 1975, 'Москва', 'teacher'),
('Семен', 'Аристархов', 'Павлович', 1980, 'Санкт-Петербург', 'teacher'),
('Иван', 'Иванов', 'Иванович', 1970, 'Новосибирск', 'teacher');

-- Наполнение таблицы disciplines
INSERT INTO disciplines (name, description) VALUES
('Машинное обучение', 'Курс по основам машинного обучения'),
('Проектирование ИС', 'Курс по проектированию информационных систем'),
('Системы ERP', 'Курс по основам ERP-систем');

-- Наполнение таблицы sessions
INSERT INTO sessions (discipline_id, type, speaker_id, start_time, end_time, system, room_id) VALUES
(1, 'lecture', 3, '2024-04-11 14:00', '2024-04-11 17:00', 'zoom', 230340930),
(2, 'lecture', 4, '2024-04-12 19:00', '2024-04-12 20:40', 'zoom', 236740930),
(3, 'exam', 5, '2024-04-13 18:00', '2024-04-13 21:00', 'zoom', 230340930);
"""
# Выполнение SQL-запроса
try:
    cursor.execute(insert_query)
    conn.commit()
    print("Данные успешно добавлены.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")


Данные успешно добавлены.


In [4]:
%%time
import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
cursor.execute("SELECT * FROM users;")
rows = cursor.fetchall()
display(rows)


[(1, 'Иван', 'Иванов', 'Иванович', 1998, 'Москва', 'student'),
 (2, 'Петр', 'Петров', 'Петрович', 1985, 'Саранск', 'student'),
 (3, 'Игорь', 'Павлов', 'Алексеевич', 1975, 'Москва', 'teacher'),
 (4, 'Семен', 'Аристархов', 'Павлович', 1980, 'Санкт-Петербург', 'teacher'),
 (5, 'Иван', 'Иванов', 'Иванович', 1970, 'Новосибирск', 'teacher')]

CPU times: user 15.8 ms, sys: 1.72 ms, total: 17.5 ms
Wall time: 68.3 ms


In [17]:

# Добавление данных в таблицы students_sessions, videos, video_annotations, materials,material_annotations
insert_query = """
-- Наполнение таблицы students_sessions
INSERT INTO students_sessions (student_id, session_id) VALUES
(1, 1),
(2, 1),
(1, 2),
(2, 3);

-- Наполнение таблицы videos
INSERT INTO videos (session_id, file_path) VALUES
(1, '/videos/ml_lecture_20240411.mp4'),
(2, '/videos/is_lecture_20240412.mp4'),
(3, '/videos/erp_exam_20240413.mp4');

-- Наполнение таблицы video_annotations
INSERT INTO video_annotations (video_id, timestamp, content) VALUES
(1, '00:15:00', 'Обсуждение алгоритма регрессии'),
(2, '00:05:00', 'Введение в проектирование ИС'),
(3, '00:30:00', 'Обсуждение основ ERP-систем');

-- Наполнение таблицы materials
INSERT INTO materials (session_id, file_path) VALUES
(1, '/materials/ml_slides.pdf'),
(2, '/materials/is_slides.pdf'),
(3, '/materials/erp_slides.pdf');

-- Наполнение таблицы material_annotations
INSERT INTO material_annotations (material_id, page_number, content) VALUES
(1, 5, 'Пример линейной регрессии'),
(2, 3, 'Диаграмма классов'),
(3, 7, 'Модуль управления финансами');
"""
# Выполнение SQL-запроса
try:
    cursor.execute(insert_query)
    conn.commit()
    print("Данные успешно добавлены.")
except Exception as e:
    print(f"Ошибка выполнения запроса: {e}")


Данные успешно добавлены.


In [5]:

import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
# Запрос 1: Все лекции, которые начинаются через 5 минут
#("SELECT * FROM sessions WHERE type = 'lecture' AND start_time BETWEEN NOW() AND NOW() + INTERVAL '5 minutes';")
cursor.execute("SELECT * FROM sessions WHERE type = 'lecture' AND start_time BETWEEN '2024-04-12 18:55' AND NOW() + INTERVAL '5 minutes';")
rows = cursor.fetchall()
display(rows)


[(2,
  2,
  'lecture',
  4,
  datetime.datetime(2024, 4, 12, 19, 0),
  datetime.datetime(2024, 4, 12, 20, 40),
  'zoom',
  236740930)]

In [6]:
# Запрос 2: Расписание 1 курса на следующую неделю
#import psycopg2
conn = psycopg2.connect(connection_string)
cursor = conn.cursor()
cursor.execute("SELECT s.* FROM sessions s \
              JOIN students_sessions ss ON s.id = ss.session_id \
              JOIN users u ON ss.student_id = u.id \
              WHERE u.role = 'student' AND u.id = 1 \
              AND start_time BETWEEN NOW() - INTERVAL '1 year' AND NOW() + INTERVAL '2 weeks';")
rows = cursor.fetchall()
display(rows)
cursor.close()
conn.commit()
conn.close()

[(1,
  1,
  'lecture',
  3,
  datetime.datetime(2024, 4, 11, 14, 0),
  datetime.datetime(2024, 4, 11, 17, 0),
  'zoom',
  230340930),
 (2,
  2,
  'lecture',
  4,
  datetime.datetime(2024, 4, 12, 19, 0),
  datetime.datetime(2024, 4, 12, 20, 40),
  'zoom',
  236740930)]

## Перевод заполнения таблиц импортом из JSON файлов и запросы с использованием библиотеки SqlAlchemy

### Удаляем всю ранне созданную схему базы (через pgAdmin pgsql tool )
DROP SCHEMA public CASCADE;
CREATE SCHEMA public;

In [2]:
%%capture
!pip install psycopg2 pandas sqlalchemy python-dotenv


### Настройка подключения к базе данных через SQLAlchemy

In [3]:
%%capture
import json
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, ForeignKey, TIMESTAMP, JSON
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
import os

# Загружаем переменные окружения
load_dotenv()

# Получаем параметры подключения
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Создаем подключение к базе данных
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
# Создаем сессию
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [11]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("SET search_path TO public"))
    conn.commit()


In [39]:
from sqlalchemy import Column, Integer, String, TIMESTAMP, ForeignKey, Text, DateTime
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import text

Base = declarative_base()

class Student(Base):
    __tablename__ = "students"
    id = Column(Integer, primary_key=True)
    info = Column(JSONB, nullable=False, server_default=text("'{}'")) 

class Teacher(Base):
    __tablename__ = "teachers"
    id = Column(Integer, primary_key=True)
    info = Column(JSONB, nullable=False, server_default=text("'{}'")) 

class Discipline(Base):
    __tablename__ = "disciplines"
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    extra_data = Column(JSONB, nullable=True, server_default=text("'{}'")) 

class Schedule(Base):
    __tablename__ = "schedule"
    id = Column(Integer, primary_key=True)
    discipline_id = Column(Integer, ForeignKey("disciplines.id")) 
    teacher_id = Column(Integer, ForeignKey("teachers.id")) 
    type = Column(String(50), nullable=False)
    start_time = Column(TIMESTAMP, nullable=False)
    end_time = Column(TIMESTAMP, nullable=False)
    extra_data = Column(JSONB, nullable=True, server_default=text("'{}'")) 

# Создание всех таблиц
Base.metadata.create_all(engine)
print("Все таблицы успешно созданы.")

/var/folders/gn/lhwr98lx5hd671qc72znvtjr0000gn/T/ipykernel_1523/293106250.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Все таблицы успешно созданы.


In [ ]:
Base.metadata.drop_all(engine)  # Удалил старые таблицы

## Пересоздание основных и дополнительных таблиц базы данных с помощью SQLAlchemy

In [16]:
from sqlalchemy import Column, Integer, String, TIMESTAMP, ForeignKey, Text, DateTime
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import text
from sqlalchemy.orm import relationship

Base = declarative_base()

class Student(Base):
    __tablename__ = "students"
    id = Column(Integer, primary_key=True)
    info = Column(JSONB, nullable=False, server_default=text("'{}'"))
    questions = relationship("StudentQuestion", back_populates="student")

class Teacher(Base):
    __tablename__ = "teachers"
    id = Column(Integer, primary_key=True)
    info = Column(JSONB, nullable=False, server_default=text("'{}'"))

class Discipline(Base):
    __tablename__ = "disciplines"
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    extra_data = Column(JSONB, nullable=True, server_default=text("'{}'"))
    video_records = relationship("VideoRecord", back_populates="discipline")
    presentations = relationship("Presentation", back_populates="discipline")

class Schedule(Base):
    __tablename__ = "schedule"
    id = Column(Integer, primary_key=True)
    discipline_id = Column(Integer, ForeignKey("disciplines.id"))
    teacher_id = Column(Integer, ForeignKey("teachers.id"))
    type = Column(String(50), nullable=False)
    start_time = Column(TIMESTAMP, nullable=False)
    end_time = Column(TIMESTAMP, nullable=False)
    extra_data = Column(JSONB, nullable=True, server_default=text("'{}'"))

class VideoRecord(Base):
    __tablename__ = "video_records"
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    file_path = Column(String, nullable=False)
    duration = Column(Integer, nullable=False)  # в секундах
    discipline_id = Column(Integer, ForeignKey("disciplines.id"), nullable=False)
    extra_data = Column(JSONB, server_default=text("'{}'"))
    
    discipline = relationship("Discipline", back_populates="video_records")
    questions = relationship("StudentQuestion", back_populates="video_record")

class Presentation(Base):
    __tablename__ = "presentations"
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    file_path = Column(String, nullable=False)
    discipline_id = Column(Integer, ForeignKey("disciplines.id"), nullable=False)
    extra_data = Column(JSONB, server_default=text("'{}'"))
    
    discipline = relationship("Discipline", back_populates="presentations")

class StudentQuestion(Base):
    __tablename__ = "student_questions"
    id = Column(Integer, primary_key=True)
    # text = Column(Text, nullable=False)
    timestamp = Column(DateTime, nullable=False)
    student_id = Column(Integer, ForeignKey("students.id"), nullable=False)
    video_id = Column(Integer, ForeignKey("video_records.id"), nullable=False)
    extra_data = Column(JSONB, server_default=text("'{}'"))
    
    student = relationship("Student", back_populates="questions")
    video_record = relationship("VideoRecord", back_populates="questions")

# Создание всех таблиц
Base.metadata.create_all(engine)
print("Все 7 таблиц успешно созданы.")

/var/folders/gn/lhwr98lx5hd671qc72znvtjr0000gn/T/ipykernel_2527/2189783243.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Все 7 таблиц успешно созданы.


In [21]:
# Удаляем все записи из таблиц перед повторной вставкой
# Откатываем сессию, если в ней есть незавершенные транзакции

from sqlalchemy.orm import sessionmaker
from sqlalchemy import text  # Добавляем явное объявление SQL-запроса
session.rollback()

# Удаляем все записи из таблиц перед вставкой
session.execute(text("DELETE FROM schedule"))
session.execute(text("ALTER SEQUENCE schedule_id_seq RESTART WITH 1"))
session.commit()
session.execute(text("DELETE FROM disciplines"))
session.execute(text("ALTER SEQUENCE disciplines_id_seq RESTART WITH 1"))
session.commit()
session.execute(text("DELETE FROM students"))
session.execute(text("ALTER SEQUENCE students_id_seq RESTART WITH 1"))
session.commit()
session.execute(text("DELETE FROM teachers"))
session.execute(text("ALTER SEQUENCE teachers_id_seq RESTART WITH 1"))
session.commit()


### Загрузка данных в таблицы с помощью импорта из json -файлов

In [22]:
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
from models import (
    Base, Student, Teacher, Discipline, 
    Schedule, VideoRecord, Presentation, StudentQuestion
)

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()


def load_json_data(filename):
    """Загружает данные из JSON-файла."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            return json.load(file)
    except Exception as e:
        print(f"Ошибка загрузки {filename}: {e}")
        return []

def check_foreign_keys(table_name, data, foreign_keys):
    """Проверяет существование внешних ключей."""
    for entry in data:
        for key, ref_table in foreign_keys.items():
            if not session.query(ref_table).get(entry[key]):
                print(f"В {table_name} запись {entry} содержит несуществующий {key}={entry[key]}")
                return False
    return True

#  Загрузка данных 
try:
    # 1. Студенты
    students_data = load_json_data("students.json")
    for student in students_data:
        session.add(Student(info=student["info"]))
    session.commit()
    print("Студенты загружены.")

    # 2. Преподаватели
    teachers_data = load_json_data("teachers.json")
    for teacher in teachers_data:
        session.add(Teacher(info=teacher["info"]))
    session.commit()
    print("Преподаватели загружены.")

    # 3. Дисциплины
    disciplines_data = load_json_data("disciplines.json")
    for disc in disciplines_data:
        session.add(Discipline(name=disc["name"], extra_data=disc["extra_data"]))
    session.commit()
    print("Дисциплины загружены.")

    # 4. Расписание (schedule)
    schedule_data = load_json_data("schedule.json")
    # Проверка времени и внешних ключей
    valid = True
    for entry in schedule_data:
        if entry["start_time"] >= entry["end_time"]:
            print(f"Некорректное время в расписании: {entry}")
            valid = False
    if valid and check_foreign_keys("schedule", schedule_data, {"discipline_id": Discipline, "teacher_id": Teacher}):
        for entry in schedule_data:
            session.add(Schedule(**entry))
        session.commit()
        print("Расписание загружено.")

    # 5. Видеозаписи
    video_data = load_json_data("video_records.json")
    if check_foreign_keys("video_records", video_data, {"discipline_id": Discipline}):
        for video in video_data:
            session.add(VideoRecord(**video))
        session.commit()
        print("Видеозаписи загружены.")

    # 6. Презентации
    presentations_data = load_json_data("presentations.json")
    if check_foreign_keys("presentations", presentations_data, {"discipline_id": Discipline}):
        for pres in presentations_data:
            session.add(Presentation(**pres))
        session.commit()
        print("Презентации загружены.")

    # 7. Вопросы студентов
    questions_data = load_json_data("student_questions.json")
    if check_foreign_keys("student_questions", questions_data, {"student_id": Student, "video_id": VideoRecord}):
        for question in questions_data:
            session.add(StudentQuestion(**question))
        session.commit()
        print("Вопросы студентов загружены.")

except IntegrityError as e:
    session.rollback()
    print(f"Ошибка целостности данных: {e}")
except Exception as e:
    session.rollback()
    print(f"Неизвестная ошибка: {e}")
finally:
    session.close()

print("Загрузка данных завершена.")

Студенты загружены.
Преподаватели загружены.
Дисциплины загружены.
Расписание загружено.
Видеозаписи загружены.
Презентации загружены.
Вопросы студентов загружены.
Загрузка данных завершена.


/var/folders/gn/lhwr98lx5hd671qc72znvtjr0000gn/T/ipykernel_2527/263929153.py:28: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  if not session.query(ref_table).get(entry[key]):


In [18]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from models import (
    VideoRecord, Presentation, StudentQuestion, 
    Discipline, Student, Teacher, Schedule  # Импорт всех моделей
)

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

# Создаем новые таблицы
Base.metadata.create_all(
    engine,
    tables=[
        # VideoRecord.__table__,
        # Presentation.__table__,
        StudentQuestion.__table__
    ]
)

print("Таблицы успешно созданы.")

Таблицы успешно созданы.


/var/folders/gn/lhwr98lx5hd671qc72znvtjr0000gn/T/ipykernel_2527/4255242117.py:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Дозагрузка данных в расписание

In [23]:
# Подгрузим дополнительные данные в расписание 
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
from models import Schedule, Discipline, Teacher

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

def load_json_data(filename):
    """Загружает данные из JSON-файла."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            return json.load(file)
    except Exception as e:
        print(f"Ошибка загрузки {filename}: {e}")
        return []

def check_foreign_keys(data):
    """Проверяет существование discipline_id и teacher_id."""
    for entry in data:
        discipline = session.get(Discipline, entry["discipline_id"])
        teacher = session.get(Teacher, entry["teacher_id"])
        if not discipline:
            print(f"Некорректный discipline_id={entry['discipline_id']} в записи {entry}")
            return False
        if not teacher:
            print(f"Некорректный teacher_id={entry['teacher_id']} в записи {entry}")
            return False
    return True

# Загрузка данных из schedule_update.json
schedule_update_data = load_json_data("schedule_update.json")

if schedule_update_data:
    try:
        # Проверка времени и внешних ключей
        valid = True
        for entry in schedule_update_data:
            if entry["start_time"] >= entry["end_time"]:
                print(f"Некорректное время: {entry}")
                valid = False
        
        if valid and check_foreign_keys(schedule_update_data):
            for entry in schedule_update_data:
                session.add(Schedule(**entry))
            session.commit()
            print("Данные из schedule_update.json успешно добавлены в расписание.")
    except IntegrityError as e:
        session.rollback()
        print(f"Ошибка целостности: {e}")
    except Exception as e:
        session.rollback()
        print(f"Неизвестная ошибка: {e}")
    finally:
        session.close()
else:
    print("Файл schedule_update.json пуст или содержит ошибки.")

Данные из schedule_update.json успешно добавлены в расписание.


## Запрос 3: Все лекции, которые начинаются через 5 минут ()

In [81]:
from datetime import datetime, timedelta

# Получаем текущее время
current_time = datetime.now()

# SQL-запрос
query = text("""
    SELECT id, discipline_id, teacher_id, type, start_time, end_time, extra_data->>'course' AS course 
    FROM schedule 
    WHERE type = 'lecture' 
    AND start_time BETWEEN :current_time AND :current_time_plus_5
""")

# Выполняем запрос
result = session.execute(query, {
    "current_time": current_time,
    "current_time_plus_5": current_time + timedelta(days=5) # minutes=5
}).fetchall()

# Выводим результат
for row in result:
    print(row)


## Запрос 4: Расписание 1 курса на следующую неделю

In [26]:
from sqlalchemy import and_, cast, Integer
from sqlalchemy.sql import func
from datetime import datetime, timedelta
from models import Schedule

# Определение дат следующей недели
current_date = datetime(2025, 3, 28)  # Пример текущей даты
next_week_start = current_date + timedelta(days=(7 - current_date.weekday()))
next_week_end = next_week_start + timedelta(days=6)

# Запрос к базе данных
schedule_query = (
    session.query(Schedule)
    .filter(
        and_(
            # Фильтр по курсу (значение в JSONB "course" = 1)
            cast(Schedule.extra_data["course"].astext, Integer) == 1,
            # Фильтр по дате (следующая неделя)
            Schedule.start_time.between(next_week_start, next_week_end)
        )
    )
    .order_by(Schedule.start_time)
    .all()
)

# Вывод результатов
print("Расписание для 1 курса на следующую неделю:")
for entry in schedule_query:
    print(
        f"Дисциплина: {entry.discipline_id}, "
        f"Тип: {entry.type}, "
        f"Начало: {entry.start_time}, "
        f"Тема: {entry.extra_data.get('topic')}"
    )

Расписание для 1 курса на следующую неделю:
Дисциплина: 1, Тип: lecture, Начало: 2025-03-31 19:00:00, Тема: Основы машинного обучения
Дисциплина: 2, Тип: practice, Начало: 2025-03-31 19:00:00, Тема: Программирование на Python


## 5. Контекстный поиск.
### Все позиции в видеозаписях с упоминанием термина "корутина"

In [58]:
from sqlalchemy import func, text
from models import VideoRecord

# Создание GIN индекса для ускорения поиска 
session.execute(text("""
    CREATE INDEX IF NOT EXISTS idx_video_records_extra_data 
    ON public.video_records 
    USING GIN ((extra_data->'keywords'), (extra_data->'transcript'));
"""))
session.commit()

# Запрос
query = (
    session.query(VideoRecord)
    .filter(
        func.to_tsvector('russian', VideoRecord.extra_data['transcript'].astext)
        .op('@@')(func.to_tsquery('корутина'))
        | VideoRecord.extra_data['keywords'].contains(['корутина'])
    )
    .all()
)

# Вывод
print("Видеозаписи с упоминанием 'корутина':")
for video in query:
    print(f"- {video.title} (ID: {video.id})")

Видеозаписи с упоминанием 'корутина':
- Лекция 1: Введение в машинное обучение (ID: 1)


In [52]:

# Откатываем сессию, если в ней есть незавершенные транзакции

from sqlalchemy.orm import sessionmaker
from sqlalchemy import text  # Добавляем явное объявление SQL-запроса
session.rollback()
session.execute(text("""
    DROP INDEX IF EXISTS idx_presentations_keywords;
    DROP INDEX IF EXISTS idx_presentations_extra_data;
"""))

session.execute(text("""
    CREATE INDEX IF NOT EXISTS idx_presentations_keywords 
    ON public.presentations 
    USING GIN ((extra_data->'keywords'));
"""))
session.commit()

In [82]:
from sqlalchemy import func, text
from models import Presentation
# Поиск презентаций
phrases = ["антипаттерн", "LLM", "нейронные сети"]

query = (
    session.query(Presentation)
    .filter(
        # Поиск по ключевым словам
        Presentation.extra_data['keywords'].contains(phrases)
        |
        # Полнотекстовый поиск по транскрипту
        func.to_tsvector('russian', Presentation.extra_data['transcript'].astext)
        .op("@@")(func.to_tsquery(' | '.join([f"'{p}'" for p in phrases])))
    )
    .all()
)

# Вывод
print("Результаты поиска:")
for pres in query:
    print(f"- {pres.title} (ID: {pres.id})")

Результаты поиска:
- Основы ML (ID: 4)
- Python для начинающих (ID: 5)


## 6. Контекстный поиск. 
### Все элементы раздаточного материала, во время демонстрации которых произносились слова "антипаттерн", "не делайте так", "плохой пример"


### Догрузка данных в таблицу презентаций для контекстного поиска

In [57]:
import json
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
from models import (
    Base, Student, Teacher, Discipline, 
    Schedule, VideoRecord, Presentation, StudentQuestion
)

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

# Удаление всех записей из таблицы presentations
session.execute(text("DELETE FROM public.presentations"))
session.commit()

def load_json_data(filename):
    """Загружает данные из JSON-файла."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            return json.load(file)
    except Exception as e:
        print(f"Ошибка загрузки {filename}: {e}")
        return []


def check_foreign_keys(table_name, data, foreign_keys):
    """Проверяет существование внешних ключей."""
    for entry in data:
        for key, ref_table in foreign_keys.items():
            if not session.query(ref_table).get(entry[key]):
                print(f"В {table_name} запись {entry} содержит несуществующий {key}={entry[key]}")
                return False
    return True
try:
    # 6. Презентации
    presentations_data = load_json_data("presentations.json")
    if check_foreign_keys("presentations", presentations_data, {"discipline_id": Discipline}):
        for pres in presentations_data:
            session.add(Presentation(**pres))
        session.commit()
        print("Презентации загружены.")

except IntegrityError as e:
    session.rollback()
    print(f"Ошибка целостности данных: {e}")
except Exception as e:
    session.rollback()
    print(f"Неизвестная ошибка: {e}")
finally:
    session.close()

print("Загрузка данных завершена.")

Презентации загружены.
Загрузка данных завершена.


/var/folders/gn/lhwr98lx5hd671qc72znvtjr0000gn/T/ipykernel_2527/732590196.py:33: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  if not session.query(ref_table).get(entry[key]):


In [59]:
# Проверка данных
presentations = session.query(Presentation).all()
for pres in presentations:
    print(f"Title: {pres.title}")
    print(f"Transcript: {pres.extra_data.get('transcript', 'N/A')}\n")

Title: Основы ML
Transcript: LLM (Large Language Models) — это современные архитектуры, основанные на нейронных сетях. Не делайте так: использование устаревших методов может быть плохим примером.

Title: Python для начинающих
Transcript: Использование глобальных переменных — это антипаттерн. Плохой пример: не делайте так, это нарушает принципы чистого кода.

Title: Оптимизация баз данных
Transcript: Создание индексов — важный этап оптимизации. Не делайте так: отсутствие индексов — плохой пример проектирования.



In [68]:
from sqlalchemy import func
from models import Presentation

session.rollback()
phrases = ["антипаттерн", "не делайте так", "плохой пример"]

# Формируем TSQUERY
tsquery = ' | '.join([f"'{p}'" for p in phrases])

query = (
    session.query(Presentation)
    .filter(
        func.to_tsvector('russian', Presentation.extra_data['transcript'].astext)
        .op("@@")(func.to_tsquery(tsquery))
    )
    .all()  # Метод all() вызывается после закрытия filter()
)

print("Результаты поиска по транскрипту:")
for pres in query:
    print(f"- {pres.title} (ID: {pres.id})")

📊 Результаты поиска по транскрипту:
- Основы ML (ID: 4)
- Python для начинающих (ID: 5)
- Оптимизация баз данных (ID: 6)


## Семантический поиск. 
Догрузка транскриптов по видеолекциям, с упоминаниями различий блочных, символьных устройств и обучение в контексте LLM

In [71]:
# загрузка тестовых данных для выполнения запроса 
import json
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
from models import (
    Base, Student, Teacher, Discipline, 
    Schedule, VideoRecord, Presentation, StudentQuestion
)
session.rollback()
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

# Удаление всех записей из таблицы video_records
# session.execute(text("DELETE FROM public.video_records"))
# session.commit()

def load_json_data(filename):
    """Загружает данные из JSON-файла."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            return json.load(file)
    except Exception as e:
        print(f"Ошибка загрузки {filename}: {e}")
        return []


def check_foreign_keys(table_name, data, foreign_keys):
    """Проверяет существование внешних ключей."""
    for entry in data:
        for key, ref_table in foreign_keys.items():
            if not session.query(ref_table).get(entry[key]):
                print(f"В {table_name} запись {entry} содержит несуществующий {key}={entry[key]}")
                return False
    return True
try:
    # 5. Видеозаписи
    video_data = load_json_data("video_records_update.json")
    if check_foreign_keys("video_records", video_data, {"discipline_id": Discipline}):
        for video in video_data:
            session.add(VideoRecord(**video))
        session.commit()
        print("Видеозаписи загружены.")

except IntegrityError as e:
    session.rollback()
    print(f"Ошибка целостности данных: {e}")
except Exception as e:
    session.rollback()
    print(f"Неизвестная ошибка: {e}")
finally:
    session.close()

print("Загрузка данных завершена.")

/var/folders/gn/lhwr98lx5hd671qc72znvtjr0000gn/T/ipykernel_2527/3314554231.py:34: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  if not session.query(ref_table).get(entry[key]):


Видеозаписи загружены.
Загрузка данных завершена.


## 7. Семантический поиск.
### Все места в видеолекциях, где упоминается обучение в контексте LLM

In [11]:
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String, JSON
from models import (
    Base, Student, Teacher, Discipline, 
    Schedule, VideoRecord, Presentation, StudentQuestion
)

# Подключение к базе данных
session.rollback()
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

# Выполнение полнотекстового поиска
try:
   
    search_query = 'блочный & символьный & различие' #'чем символьное устройство отличается от блочного'
    
    # Создаем запрос
    results = (
        session.query(VideoRecord)
        .filter(
            func.to_tsvector('russian', VideoRecord.extra_data['transcript'].astext)
            .op('@@')(func.websearch_to_tsquery('russian', search_query))
        )
        .all()
    )
    
    # Выводим результаты
    if results:
        print(f"Найдено {len(results)} видеолекций:")
        for idx, record in enumerate(results, 1):
            print(f"{idx}. {record.title}")
            print(f"   Транскрипт: {record.extra_data['transcript'][:50]}...")
    else:
        print("Совпадений не найдено")
        
except Exception as e:
    session.rollback()
    print(f"Ошибка при выполнении запроса: {e}")
finally:
    session.close()

Найдено 1 видеолекций:
1. Семинар: Оптимизация SQL-запросов
   Транскрипт: Рассмотрим различия между символьными и блочными у...


## 8. Семантический поиск. 
### Ответ на вопрос: чем символьное устройство отличается от блочного?

In [6]:
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String, JSON
from models import (
    Base, Student, Teacher, Discipline, 
    Schedule, VideoRecord, Presentation, StudentQuestion
)

# Подключение к базе данных
session.rollback()
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

# Выполнение полнотекстового поиска
try:
   
    search_query = 'LLM & обучение'
    
    # Создаем запрос
    results = (
        session.query(VideoRecord)
        .filter(
            func.to_tsvector('russian', VideoRecord.extra_data['transcript'].astext)
            .op('@@')(func.websearch_to_tsquery('russian', search_query))
        )
        .all()
    )
    
    # Выводим результаты
    if results:
        print(f"Найдено {len(results)} видеолекций:")
        for idx, record in enumerate(results, 1):
            print(f"{idx}. {record.title}")
            print(f"   Транскрипт: {record.extra_data['transcript'][:50]}...")
    else:
        print("Совпадений не найдено")
        
except Exception as e:
    session.rollback()
    print(f"Ошибка при выполнении запроса: {e}")
finally:
    session.close()

Найдено 4 видеолекций:
1. Лекция 1: Введение в машинное обучение
   Транскрипт: Обучение LLM (Large Language Models) включает пред...
2. Практика: Тонкая настройка LLM
   Транскрипт: В этом видео мы рассмотрим этапы тонкой настройки ...
3. Семинар: Оптимизация LLM
   Транскрипт: Обучение LLM требует оптимизации вычислительных ре...
4. Лекция 4: Этические аспекты LLM
   Транскрипт: Обучение LLM на предвзятых данных может привести к...


### Подключение к API OpenAI для генерации embeddings:

In [ ]:
import os
from dotenv import load_dotenv

# Загружаем переменные окружения из .env
load_dotenv()

# Получаем API-ключ
api_key = os.getenv("OPENAI_API_KEY")

# Проверяем, что ключ загружен
if api_key:
    print("API Key загружен успешно.")
else:
    print("Ошибка: API Key не найден.")


API Key загружен успешно.
